# Clasificación de frases chequeables usando Bert

Para este problema se intentará usar [Beto](https://github.com/dccuchile/beto) y HuggingFace. Especialmente se sigue este [ejemplo](https://huggingface.co/transformers/custom_datasets.html#seq-imdb)

In [1]:
#!pip install transformers
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
#!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
#!tar -xzvf pytorch_weights.tar.gz
#!mv config.json pytorch/.
#!mv vocab.txt pytorch/.

In [2]:
#!pip install torch torchvision

In [3]:
import torch
from transformers import BertForMaskedLM, BertTokenizer

In [8]:
tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("pytorch/")
#model.eval()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = "[CLS] La [MASK] es el problema de nuestro [MASK] [SEP]"
masked_indxs = (2,8)

tokens = tokenizer.tokenize(text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens])

predictions = model(tokens_tensor)[0]

for i,midx in enumerate(masked_indxs):
    idxs = torch.argsort(predictions[0,midx], descending=True)
    predicted_token = tokenizer.convert_ids_to_tokens(idxs[:5])
    print('MASK',i,':',predicted_token)

MASK 0 : ['ignorancia', 'guerra', 'pobreza', 'muerte', 'violencia']
MASK 1 : ['país', 'tiempo', 'pueblo', 'mundo', 'planeta']


In [20]:
import glob
import pickle
import sys

import numpy as np
import pandas as pd

from constants import POS_TAGGED_FOLDER


def get_tagged_sentences(folder):
    # Load all the tagged sentences included in the .pickle files 
    parsed_sentences = []
    for filename in glob.glob(folder + '*.pickle'):
        with open(filename, 'rb') as tagged_file:
            parsed_sentences = parsed_sentences + pickle.load(tagged_file, encoding="bytes")
    return parsed_sentences

tagged_sentences = get_tagged_sentences(POS_TAGGED_FOLDER)
data = [{'sentence': item[b'sentence'].decode('utf8').lower(), 'target': item[b'classification'].decode('utf8')} for item in tagged_sentences]
df = pd.DataFrame(data)

In [3]:
data[0]

{'sentence': '\ufeffdiscurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo de sesiones ordinarias del congreso nacional',
 'target': 'non-fact-checkable'}

In [4]:
df.head()

,sentence,target
0,﻿discurso de la presidenta cristina fernández ...,non-fact-checkable
1,muy buenos días a todos y todas,non-fact-checkable
2,vengo una vez más a cumplir con lo dispuesto p...,non-fact-checkable
3,y la verdad que quiero comenzar dando cuenta d...,non-fact-checkable
4,el día viernes 27 pude leer un tuit en la cuen...,non-fact-checkable


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
'''
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['target']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

train_texts = strat_train_set.drop(['target'], axis=1).astype(str)
train_labels = strat_train_set['target'].values.tolist()

test_texts = strat_test_set.drop(['target'], axis=1).values.tolist()
test_labels = strat_test_set['target'].values.tolist()

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)
'''
###
X = [e['sentence'] for e in data]
y = [0  if e['target'] == 'fact-checkable' else 1 for e in data]

sentences = ["[CLS] " + e['sentence'] + " [SEP]" for e in data]
labels = [0  if e['target'] == 'fact-checkable' else 1 for e in data]

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=.2)

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

NameError: name 'data' is not defined

In [12]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [13]:
test_texts[:5]

['¿por qué protestan? les digo',
 'por eso el gran desafío fue mantener políticas públicas activas y aumentar las que ya habíamos desplegado para precisamente poder superar toda esta crisis',
 'stolbizer- creo posible y necesario establecer acuerdos',
 ' la república argentina es el único país que ha descendido en forma negativa su deuda externa en todo el mundo',
 'este adelanto en materia de transplante es también patrimonio del parlamento argentino a partir de la aprobación de la ley del donante presunto']

In [14]:
import torch

class ChqDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ChqDataset(train_encodings, train_labels)
val_dataset = ChqDataset(val_encodings, val_labels)
test_dataset = ChqDataset(test_encodings, test_labels)

In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [2]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

In [18]:

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = BertForSequenceClassification.from_pretrained("pytorch/")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pytorch/

Step,Training Loss
10,0.680541
20,0.680453
30,0.620086
40,0.559473
50,0.583552
60,0.521176
70,0.452430
80,0.367741
90,0.369752
100,0.361647


TrainOutput(global_step=775, training_loss=0.1933819776965726)

In [15]:
trainer.save_model('qsch')

In [19]:
trainer.evaluate()

{'eval_loss': 0.46729370951652527,
 'eval_accuracy': 0.9142394822006472,
 'eval_f1': 0.9284750337381916,
 'eval_precision': 0.9373297002724795,
 'eval_recall': 0.9197860962566845,
 'epoch': 5.0}

In [50]:
demo_texts = ['hay más de 25 millones de trabajadores que están registrados', 
              'hay más de 50 millones de trabajadores que están registrados', 
              'discurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo',
             'vamos a otorgar un subsidio de 15 mil pesos por cada nuevo empleo registrado durante doce meses',
              'prometo bajar la inflación a un 10% en el 2021',
             'se le saca a la Ciudad el 12% del presupuesto',
             ]
demo_labels = [0, 0, 1, 1, 0, 0]

In [51]:
demo_encodings = tokenizer(demo_texts, truncation=True, padding=True)

In [52]:
len(demo_encodings.values())

3

In [53]:
demo_dataset = ChqDataset(demo_encodings, demo_labels)

In [54]:
trainer.predict(demo_dataset)

PredictionOutput(predictions=array([[ 4.438871 , -3.965041 ],
       [ 4.4410357, -3.9678428],
       [-3.9221334,  3.6707048],
       [ 2.477028 , -1.8974282],
       [ 3.7844305, -3.4295647],
       [ 4.4204555, -3.960571 ]], dtype=float32), label_ids=array([0, 0, 1, 1, 0, 0]), metrics={'eval_loss': 0.7314813137054443, 'eval_accuracy': 0.8333333333333334, 'eval_f1': 0.6666666666666666, 'eval_precision': 1.0, 'eval_recall': 0.5})

In [20]:
demo_texts

['hay más de 25 millones de trabajadores que están registrados',
 'hay más de 50 millones de trabajadores que están registrados',
 'discurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo',
 'vamos a otorgar un subsidio de 15 mil pesos por cada nuevo empleo registrado durante doce meses',
 'se le saca a la Ciudad el 12% del presupuesto']

In [21]:
#without labels
class ChqInferenceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, total_items):
        self.encodings = encodings
        self.total_items = total_items

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #item['labels'] = torch.tensor(self.labels[idx])
        print(item)
        return item

    def __len__(self):
        return self.total_items

In [22]:
demo_encodings = tokenizer(demo_texts, truncation=True, padding=True)
nolabel_dataset = ChqInferenceDataset(demo_encodings, len(demo_texts))
output = trainer.predict(nolabel_dataset)

NameError: name 'trainer' is not defined

In [121]:
output.predictions

array([[ 3.3871017, -3.7961414],
       [ 3.3651884, -3.7773092],
       [-3.2432277,  3.7316933],
       [-1.8270873,  3.0422065],
       [ 3.815342 , -4.118779 ]], dtype=float32)

## Sección de inferencia a partir de la carga del modelo pre-entrenado

In [4]:
from transformers import pipeline

In [5]:
text_classification = pipeline(
    "sentiment-analysis",
    model="./qsch",
    tokenizer="dccuchile/bert-base-spanish-wwm-cased"
)

In [6]:
eval_claims = [
    'hay más de 25 millones de trabajadores que están registrados', 
    'hay más de 50 millones de trabajadores que están registrados', 
    'discurso de la presidenta cristina fernández de kirchner en la inauguracion del 133° periodo',
    'vamos a otorgar un subsidio de 15 mil pesos por cada nuevo empleo registrado durante doce meses',
    'prometo bajar la inflación a un 10% en el 2021',
    'se le saca a la Ciudad el 12% del presupuesto',
    'no es cierto que vamos a devaluar en enero',
    'Guzmán está tomando deuda al 17%',
    'La única posibilidad de Argentina de iniciar la recuperación es crear las condiciones para impulsar la inversión, generadora de nuevos puestos de trabajo bajo regímenes más modernos y tan justos como los vigentes',
    'Y cuando decimos que queremos bajar el déficit a cuatro puntos y medio –de los 9 que seguramente tendremos este año–, nos llaman ajustadores',
    'Hoy se consume la misma energía para producción industrial que se consumía el 19 de marzo. Y el consumo es más alto',
    'Yo no comparto la mirada de muchos que dicen que esto es un ajuste sobre los jubilados porque en verdad se basa en dos puntos: la variación de los salarios y la recaudación, que es importante porque si no recaudás, no podemos pagar',
    'Alberto qué recuerdos tienes de la última vez que te visitó Diego, en enero y cuál es el recuerdo del primer contacto que tenés con Diego.',
    'Yo creo que, en verdad, todos los argentinos estamos en deuda con él, porque son esos hombres que… bueno con vos pasa un poco lo mismo, son esos hombres que sólo nos han dado alegría. ',
    '¿Cómo se hace – Alberto - para convivir siendo el presidente de todos los argentinos, el hincha, el hombre común, en un momento para desempeñar tantos roles, porque por un lado hay una cosa del protocolo, pero surge el hincha, la emoción, del tipo común, cómo se hace para manejar eso?',
]
text_classification(eval_claims)

[{'label': 'LABEL_0', 'score': 0.9992413520812988},
 {'label': 'LABEL_0', 'score': 0.9992098212242126},
 {'label': 'LABEL_1', 'score': 0.9990658760070801},
 {'label': 'LABEL_1', 'score': 0.9923797249794006},
 {'label': 'LABEL_0', 'score': 0.9988777041435242},
 {'label': 'LABEL_0', 'score': 0.9996417760848999},
 {'label': 'LABEL_1', 'score': 0.9501869082450867},
 {'label': 'LABEL_0', 'score': 0.9963929057121277},
 {'label': 'LABEL_1', 'score': 0.9972434639930725},
 {'label': 'LABEL_1', 'score': 0.985137403011322},
 {'label': 'LABEL_0', 'score': 0.9995606541633606},
 {'label': 'LABEL_1', 'score': 0.9986720085144043},
 {'label': 'LABEL_1', 'score': 0.999411404132843},
 {'label': 'LABEL_1', 'score': 0.9993310570716858},
 {'label': 'LABEL_1', 'score': 0.9992738366127014}]

# Bonus track: test de un modelo de QA

In [45]:
nlp = pipeline(
    'question-answering', 
    model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer=(
        'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',  
        {"use_fast": False}
    )
)

In [50]:
#https://www.youtube.com/watch?v=yA_IDTcoj-Q
#lo pasamos por el desgrabador y obtenemos el siguiente texto
context = r"""
hoy vamos a dar la lección 1 sociología y vida cotidiana vamos a repasar los principales términos de los autores de la lección 1 en primer lugar tenemos a márquez donde los términos más importantes lo que nosotros tenemos que destacar de este autor es que naturalizamos lo social es decir que tenemos una actitud irreflexiva frente a nuestras conductas cotidianas el segundo actor fue meals que nos habló de imaginación sociológica que es la invasión de imaginación sociológica bueno es una cualidad que permite comprender la relación entre problema e inquietud y entre biografía e historia he aquí que tenemos los principales términos que tenemos que identificar de este autor en lo que respecta a la sociedad la historia de la sociedad y la biografía individual lo que es una inquietud y lo que es un problema recordemos que inquietud hace referencia a un asunto privado que ocupa a una sola persona y un problema es un asunto público que refiere a varias personas por ejemplo el hecho del desempleo si una sola persona en la sociedad no tiene trabajo ese es un asunto privado es una inquietud pero si el 18 por ciento de la de las personas en edad laboral de una sociedad no tienen trabajo pasa a ser un asunto público con respecto a elías elías va a hablar sobre las dificultades en nuestro idioma para pensar sociológicamente a través de la cosificación que es una deshumanización de figuras sociales que nos lleva a la metafísica de esas figuras y que nuestro lenguaje está basado en un modelo de ciencias naturales y del pensamiento mágico mítico y basado básicamente en un modelo de voz céntrico donde las fantasías ocupan un lugar errado en la vida de los hombres y va a proponer el modelo de relaciones de interdependencia donde todos estamos conectados en la sociedad en el trabajo de la sociología debe ser ampliar y hacer más confiable la comprensión de los elementos o activos por su parte heller nos va a hablar de la vida cotidiana es decir la vida cotidiana es un conjunto de actividades que implican al mismo tiempo la reproducción individual y social es jerárquica por eso habla de un regimiento de la vida es heterogénea ya que es el ya que el medio de superación a la particularidad es la homogenización y habla sobre el papel que juega la ultra generalización en nuestra vida cotidiana a través de los juicios provisionales y los prejuicios y va a mencionar también la extra la extraña acción de la vida cotidiana que proviene de la cristalización de las características de la vida cotidiana que reducen el margen de movimiento y conciencia de los individuos va a mencionar también la ética que tiene que sirve para el veto de ciertas conductas y también para la transformación quiere decir que el ser que las personas son al mismo tiempo particular y específico el medio recordemos para de superación de la particularidad hacia la especificidad es la homogenización de la vida cotidiana con respecto a berger y lukman van a ver la sociedad desde dos puntos de vista primero la sociedad como realidad objetiva hablar de la institucionalización va a decir que el orden social es un producto humano y aparece cada vez que se da una tipificación recíproca de acciones habitual izadas por tipos de actores esto hay que recordarlo es bastante importante con respecto a la institucionalización estos son los principales temas y puntos que tocan bergara bergara y luminosos principales conceptos institucionalización roles sedimentación el legitimación por otro lado la sociedad con realidad subjetiva va a hablar sobre la socialización primaria que es la que se da en la primera etapa de la vida de la persona la socialización secundaria que es la siguiente como es el mantenimiento y transformación de la realidad y va a mencionar también algo sobre las teorías de la identidad edad 23 perspectivas sociológicas si quieres recibir el resumen completo tanto del primer parcial como del segundo o ambos para preparar el final solamente tienes que suscribirte a mi canal y dejarme un comentario para que te pueda enviar el email en formato pdf con este resumen vamos a comenzar tenemos tres auto autores en esta unidad número dos el primero de ellos es urgen que va a hablar principalmente su punto central va a ser hecho social que es un hecho social y va a partir de la de la sociología objetivista su sociología va a ser objetivista porque también porque exige que el sociólogo explique las causas externas que llevan a los individuos a actuar y que va a decir sobre un hecho social va a ser que son cosas va a hablar de corriente opinión iba a hablar de que la realidad social es independiente de la voluntad de los individuos y en estos son los principales términos de este autor con respecto al segundo autor que se nos presenta en esta unidad que es marx que se puede profundizar mucho más pero estos son sus principales términos que nos van a tomar en el examen el primero de ellos es la producción sobre la producción más simples la producción es de individuos que en sociedad va a decir que el proceso de trabajo es natural y social y su principal termino con relación la producción va a ser la plusvalía plusvalía también va a hablar sobre consumo y producción sobre distribución cambio y producción y sobre el capitalismo que va a decir que es un tipo de organización social cuyo objetivo es la acumulación de riquezas y continua reproducción de capital y que la expansión e integración constante del mercado exterior es una gran unidad económica más todo esto más el creciente aumento de la productividad del trabajo por la permanente innovación tecnológicos son los mecanismos que garantizan su continuidad ciclina e irving van a retomar el trabajo de marks y van a hablar van a mencionar precisamente estos términos van a hablar con respecto a la religión y las condiciones seculares principalmente del judaísmo y les va a decir que para evitar la discriminación y la destrucción deben emanciparse del comercio del dinero y que las relaciones de propiedad es el punto de partida de la teoría de clases beber por su parte va a hablar en esta en esta unidad principalmente de acción social la acusación es una acción en donde encendido aumentado por su sujeto está referido a la conducta de otras personas a la voluntad de otros por eso es habla de un sentido subjetivo sentido subjetivo no toda clase de contacto entre personas entre hombres tiene carácter social y la acción social relacionado también con lo que es dominación va a decir que es racional con arreglo a fines racional con arreglo a valores afectiva o tradicional y el método de la sociología comprensión es racionalista para medio ya que exige el sociólogo intérprete en sentido a partir del cual las personas orientan su acción bien vamos a esta unidad que es poder vamos a revisar a tres autores el primero de ellos al tu ser va a hablar del poder con respecto a la reproducción de las condiciones de producción primero hablar de la reproducción de los medios de producción a lo cual lo explica a vagamente y no le da demasiada importancia ya que es básicamente la obtención de los materiales para producir pero si le dan mayor importancia a la reproducción de las relaciones de producción existente y en la reproducción de las fuerzas productivas con respecto a la reproducción de las relaciones de producción existentes tenemos que decir que éstas están aseguradas por la acción de los aparatos de estado que terminan siendo relaciones de explotación y con respecto a la reproducción de las fuerzas productivas en la reproducción de la fuerza de trabajo será fuera de la empresa como a través del salario que le permite al trabajador vivir y satisfacer sus necesidades y a través de la calificación laboral esta planificación laboral son habilidades que se adquieren en el sistema educativo a través de los años y principalmente la sumisión al sistema por su parte va a decir que el estado es un aparato represivo que permite a la clase dominante asegurar su dominación sobre la clase trabajadora para someterlo al proceso de exploración de la plusvalía como a través de los aparatos represivos del estado y los y del aparato ideológico del estado los aparatos represivos son por ejemplo la policía el ejército la administración que funciona mediante la violencia de los aparatos ideológicos son por ejemplo la escuela la familia iglesia que funcionan mediante la ideología bien estos son los principales términos principales conceptos de este autor en relación al poder el segundo autor es beber el primero va a definir algunos términos que son importantes por ejemplo poder que es la imposición de la propia voluntad ya sea contra toda resistencia lo que es la dominación la disciplina y la asociación de dominación ellos son básicamente alguno de los principales términos que define ver quiero hablar son distintos tipos de dominación legítima el primero la industria racional la tradicional el carismático y nos va a decir que toda dominación requiere de un cuadro administrativo el primer tipo de dominación la dominación racional tiene ciertas características que enumera por ejemplo un ejercicio continuado sujeto a la ley la competencia la jerarquía administrativa reglas técnicas o normas y los funcionarios no son los propietarios de los medios materiales de administración y tampoco existe la apropiación de los cargos en el cuadro administrativo es un cuadro administrativo burocrático con respecto a la dominación tradicional nos va a decir que su legitimidad se basa en la santidad de ordenaciones y poderes de mando elevados de tiempos lejanos no se obedece a disposiciones sino a la persona los mandatos de las personas hoy legítimo por la fuerza la tradición y son legítimos por arbitrio libre del señor este tipo de dominación se ve básicamente monarquías y la dominación carismática en la que en la que mayor énfasis hace su estudio lo que es la que más le interesó a este autor este tipo de dominación es decirle carisma es una cualidad extraordinaria y se la considera esta para la persona que ejerce este tipo de dominación se la considera en posesión de fuerzas sobre humanas el reconocimiento se mantiene por corroboración es decir aquel que ejerce el dominio de tipo carismático debe mostrar que en verdad tiene estas fuerzas sobre humanas que generan el bienestar entro en todos los nominados el proceso de comunicaciones de carácter emotivo se opone a la dominación tradicional y racional y el carisma puro es específicamente extraño a la economía su cuadro administrativo está formado por servidores ligados a la autoridad en virtud de la lealtad personal y tiene un carácter extraordinario y revolucionario está este tipo de dominación también debe utilizarse debe racionalizarse ya que no se puede constantemente a ejercer este tipo de dominación como se racionaliza bueno por interés de la comunidad y por interés del cuadro administrativo al mantener su posición en caso de que desaparezca la persona portadora de carisma se hace una nueva búsqueda por revelación o por designación hecha por el que portaba el carisma por una designación echarle por el cuadro administrativo o se hace una búsqueda dentro de la familia en torno a los herederos de la persona carismática ya que se relaciona con la sangre fútbol por su parte cuando habla hable sobre poder nos va a decir que el sistema de poder de la monarquía excesivamente oneroso lo que hacía que el poder político fuera muy discontinuo por ello la burguesía y la monarquía instalaron una nueva forma de poder basada en el derecho donde ya no existe un solo poder sino que existen varios y el poder viene desde abajo y atraviesa toda relación social surgen las técnicas políticas como la disciplina la educación y el poder se ejerce en espacios cotidianos como la escuela una familia que se cambia la anátomo política por la videopolítica lanata como política se ejerce sobre los cuerpos en cambio la vida política se ejerce sobre la población la primera de ellas prescribe a partir de nuevas preexistentes las segundas reglas y normas fijadas y el poder deja de ser jurídico para hacer materialista va a hablar también sobre la delincuencia que tiene para este autor posee utilidad económica y el peligro interno es una de las condiciones de aceptabilidad del control vamos a ver los autores que se nos presentan en la unidad 4 el primero de ellos es le fevre con respecto a la ideología va a decirnos que es un reflejo invertido mutilado y deformado de lo real va a hablar también de la praxis donde intervienen mediante las mediante la cooperación y persuasión se convierte en el lenguaje y legitima las relaciones del poder esto es lo que nos dice con respecto a ideologías y estos son los principales conceptos que tenemos que tener en cuenta ideología y praxis y la ideología tiene un carácter doble por un lado son generales y especulativas y por otro lado representan intereses particulares con respecto al primer carácter general y especulativas en este lugar y actúa la coerción y la persuasión y se puede observar la transparencia de la ideología en segundo lugar cuando la iv de la gente presente intereses particulares se asume la parte de la debilidad humana y se ve la opacidad de la ideología el segundo autor alto observó hablar sobre ideología general e ideologías particulares va a ser esta distinción de la ideología general nos va a decir que no tiene historia que es eterna y nos va a decir que hay una garantía de que todo está bien como está con la condición de que los sujetos reconozcan lo que son y se conduzcan en consecuencia con respeto energía particular no hace que tiene una existencia material entonces podemos resumir que para el tuyo la ideología puede ser general particular que no tiene historia que es eterna y que tiene una existencia material la relación imaginaria de los individuos con sus condiciones reales de existencia y los sujetos por propia voluntad se someten al orden vigente y a la ideología dominante el tercer autor con respecto a la ideología nos va a decir que es un sistema de representación es dotado de existencia material y un papel histórico determinado la diferencia del autor anterior va a hablar sobre instituciones y sobre fenómenos mentales y va a dar ciertos rasgos de las ideologías por un lado nos va a decir que son sistemas completos globalizantes ya que ofrece una visión del mundo total y por otro nos va a decir que son deformantes ya que se ocultan ciertos aspectos de la realidad para servir mejor a unos intereses particulares o existen varios sistemas de representaciones y son estabilizantes y portadoras de esperanzas bien en esta unidad vamos a ver también a tres autores el primero de ellos es fashion que va a tener algunos conceptos y términos que nos va a presentar de manera muy desarrollada y muy completa he aquí los términos más importantes y los más destacados y va a mencionar en un principio de la universalidad de la subordinación femenina de que la ideología patriarcal agudiza las formas de dominación y también va a mencionar algunos conceptos relacionados con el feminismo para decir que es un movimiento social y político para la liberación del sexo femenino va a mencionar algunos principios comunes del feminismo va a hablar sobre la jerarquización de las dicotomías la glorificación del lado femenino ya que por un lado los hombres exaltan a las mujeres y por otro lado la degradan va a decir que se tomó como referente al lado masculino y va a mencionar algunas instituciones del patriarcado como lo son el lenguaje la familia la educación androcéntrica la maternidad forzada la historia robada la heterosexualidad obligatoria las religiones el derecho y la violencia de género por su parte mafia va a mencionar los principios del feminismo que es descriptivo prescriptivo y práctico va a mencionar también las dicotomías la sexualización del par y la jerarquización del par va a ser un breve resumen sobre la historia del feminismo en la década de los 70 se buscaba la igualdad en la década del 80 la diferencia y en la década del 90 se critica todo lo anterior y también no mencionar la epistemología del feminismo por su parte el último autor márquez va a hablar sobre la sociedad patriarcal y va a hablar del varón como el ser importante el varón es el importante va a decir que el varón más mujer es igual a varón completo por eso hablar de complementariedad en el varón nos va a decir que la identidad de género es un espíritu de cuerpo va a hablar sobre el modelo imagen refugio de información y angustia y con respecto al amor masculinidad va a hablar sobre el varón en propiedad y el varón entró en precario y también nos va a hablar de la socialidad bien estos son los principales conceptos de este autor bueno estos son los principales conceptos de los tres autores que se ven en la unidad número 5 se espero que te sirva este este resumen si quieres recibir el resumen completo para aprobar el segundo parcial el resumen para el primer parcial o los dos solamente suscríbete a mi canal y dejar un comentario con tu dirección de email para poder recibir este resumen en formato pdf y sin ningún costo si quieres cualquier otro resumen o una clase particular lo que necesites solamente suscribirte y dejarme un comentario y lo podemos coordinar espero que tengas mucha suerte
"""

In [51]:
nlp(
    {
        'question': '¿Cuáles son los distintos tipos de dominación legítima? ',
        'context': context
    }
)

/home/mfalcon/projects/chequeabot/cbot/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.24274341762065887,
 'start': 8846,
 'end': 8908,
 'answer': 'el primero la industria racional la tradicional el carismático'}

In [52]:
nlp(
    {
        'question': '¿Cúando se da la socialización primaria? ',
        'context': context
    }
)

/home/mfalcon/projects/chequeabot/cbot/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


{'score': 0.3397424519062042,
 'start': 3588,
 'end': 3626,
 'answer': 'primera etapa de la vida de la persona'}

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("mrm8488/RuPERTa-base-finetuned-pawsx-es")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/RuPERTa-base-finetuned-pawsx-es")

Some weights of the model checkpoint at mrm8488/RuPERTa-base-finetuned-pawsx-es were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
tokenizer_b = AutoTokenizer.from_pretrained("mrm8488/electricidad-base-finetuned-pawsx-es")
model_b = AutoModelForSequenceClassification.from_pretrained("mrm8488/electricidad-base-finetuned-pawsx-es")

In [13]:
classes = ["no son lo mismo", "son lo mismo"]

sequence_0 = "La inflación en 2016 fue de 50%"
sequence_1 = "En el 2016 tuvimos una inflación cercana al 50%"
sequence_2 = "La inflación es un problema que Argentina no puede solucionar"

paraphrase = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")
not_paraphrase = tokenizer.encode_plus(sequence_0, sequence_2, return_tensors="pt")

paraphrase_classification_logits = model(**paraphrase)[0]
not_paraphrase_classification_logits = model(**not_paraphrase)[0]

paraphrase_results = torch.softmax(paraphrase_classification_logits, dim=1).tolist()[0]
not_paraphrase_results = torch.softmax(not_paraphrase_classification_logits, dim=1).tolist()[0]

print("Should be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(paraphrase_results[i] * 100)}%")

print("\nShould not be paraphrase")
for i in range(len(classes)):
    print(f"{classes[i]}: {round(not_paraphrase_results[i] * 100)}%")

Should be paraphrase
no son lo mismo: 19%
son lo mismo: 81%

Should not be paraphrase
no son lo mismo: 66%
son lo mismo: 34%


In [21]:
claims_pair = tokenizer.encode_plus(sequence_0, sequence_1, return_tensors="pt")
claims_pair_logits = model(**claims_pair)[0]
claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]

In [15]:
for i in range(len(classes)):
    print(f"{classes[i]}: {round(claims_pair_results[i] * 100)}%")

no son lo mismo: 19%
son lo mismo: 81%


In [22]:
claims_pair_results

[0.18940554559230804, 0.8105944991111755]

In [16]:
claims = [
    'La inflación en 2016 fue de 50%',
    'En el 2016 tuvimos una inflación cercana al 50%',
    'La inflación es un problema que Argentina no puede solucionar',
    'En los 12 últimos meses la inflación fue la más baja de los últimos siete años.',
    'Hay que trabajar medio año para el Estado.',
    'Trabajamos la mitad del año para pagarle impuestos al Estado',
]

In [27]:
similar_claims = []
for i in range(len(claims)):
    for j in range(len(claims)):
        if i == j:
            continue
        claims_pair = tokenizer.encode_plus(claims[i], claims[j], return_tensors="pt")
        claims_pair_logits = model(**claims_pair)[0]
        claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]
        if claims_pair_results[1] > 0.7:
            similar_claims.append([claims[i], claims[j], claims_pair_results[1]])

In [ ]:
similar_claims_b = []
for i in range(len(claims)):
    for j in range(len(claims)):
        if i == j:
            continue
        claims_pair = tokenizer_b.encode_plus(claims[i], claims[j], return_tensors="pt")
        claims_pair_logits = model_b(**claims_pair)[0]
        claims_pair_results = torch.softmax(claims_pair_logits, dim=1).tolist()[0]
        if claims_pair_results[1] > 0.7:
            similar_claims_b.append([claims[i], claims[j], claims_pair_results[1]])

In [28]:
similar_claims

[['La inflación en 2016 fue de 50%',
  'En el 2016 tuvimos una inflación cercana al 50%',
  0.8105944991111755],
 ['En el 2016 tuvimos una inflación cercana al 50%',
  'La inflación en 2016 fue de 50%',
  0.806519627571106],
 ['Trabajamos la mitad del año para pagarle impuestos al Estado',
  'La inflación en 2016 fue de 50%',
  0.7567902207374573],
 ['Trabajamos la mitad del año para pagarle impuestos al Estado',
  'Hay que trabajar medio año para el Estado.',
  0.721567690372467]]